In [2]:
import pandas as pd

# 1. Load the CSV file
df = pd.read_csv('SIS-JIG T-Crushing PWO gain map.csv')

# 2. & 3. Filter the dataframe for the specific IDX_Tag_Name value
#filtered_df = df[df['IDX_TagName'].str.contains('SIS-JIG.SEP.APC-J140_BIN_005C', case=True, na=False)]
#filtered_df = df[df['IDX_TagName'].str.contains('APC-J140_BIN_005C', case=True, na=False)]
filtered_df = df[df['CVAPETTAG'].str.contains('J140-BIN', case=True, na=False)]

# Display the first few rows of the filtered dataframe
filtered_df.head()

,NUMBER,CVNAME,CVAPETTAG,TYPE,MVDVNAME,MVDVAPETTAG,MVDVNUMBER,GAIN-VALUE,GAIN-TAG
0,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,MV,Fresh_Feed,PID-J130-1.PV,1.1,0.033053,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
1,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,MV,1_Vibrating_Feeder1_OP,PID-J141-LIC-G1118.OP,2.1,-0.529627,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
2,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,MV,1_Vibrating_Feeder2_OP,PID-J141-LIC-G1128.OP,2.2,-0.529627,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
3,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,MV,2_Vibrating_Feeder1_OP,PID-J142-LIC-G1118.OP,3.1,-0.490164,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
4,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,MV,2_Vibrating_Feeder2_OP,PID-J142-LIC-G1128.OP,3.2,-0.490164,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...


In [3]:
print(filtered_df.shape)

(261, 9)


In [5]:
import logging
import pandas as pd
from typing import Optional, Dict, Any, Union, Callable

# Single comprehensive mapping dictionary
GAINS_COLUMN_MAPPING: Dict[str, Union[str, Dict[str, str], Callable]] = {
    # Basic column renames
    'CVNAME': 'Variable Type',
    'CVAPETTAG': 'Variable Name',

    # Type mapping
    'TYPE': {
        'MV': 'Manipulated Variable',
        'CV': 'Controlled Variable',
        'DV': 'Disturbance Variable'
    },

    # MVDVNAME formatting
    'MVDVNAME': lambda x: x.replace('_', ' ').replace('Tert Crusher', 'Tertiary Crusher'),

    # Columns to keep as-is
    'MVDVAPETTAG': 'MVDVAPETTAG',
    'MVDVNUMBER': 'MVDVNUMBER',
    'GAIN-VALUE': 'GAIN-VALUE'
}

def format_gains_map(df: pd.DataFrame, log_level: str = 'INFO') -> Optional[pd.DataFrame]:
    """
    Format and map columns in a DataFrame according to specified rules.

    Args:
        df (pd.DataFrame): Input DataFrame with columns to be transformed
        log_level (str): Logging level ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL')

    Returns:
        pd.DataFrame: Transformed DataFrame or None if an error occurs

    Raises:
        ValueError: If required columns are missing
        TypeError: If input is not a pandas DataFrame
    """
    # Set up logging
    logging.basicConfig(
        level=getattr(logging, log_level.upper()),
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Validate input
        if not isinstance(df, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame")

        # Check for missing columns
        missing_columns = [col for col in GAINS_COLUMN_MAPPING.keys() if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        # Create a copy of the DataFrame to avoid modifying the original
        df_transformed = df.copy()
        logger.info("Created copy of input DataFrame")

        # Process each column according to its mapping type
        for col, mapping in GAINS_COLUMN_MAPPING.items():
            try:
                if isinstance(mapping, str):
                    # Simple column rename
                    if col in df_transformed.columns:
                        df_transformed = df_transformed.rename(columns={col: mapping})
                        logger.debug(f"Renamed column {col} to {mapping}")

                elif isinstance(mapping, dict):
                    # Dictionary mapping (e.g., for TYPE column)
                    if col in df_transformed.columns:
                        df_transformed[col] = df_transformed[col].map(mapping)
                        unmapped = df_transformed[col].isna()
                        if unmapped.any():
                            logger.warning(f"Found unmapped values in {col}: {df_transformed.loc[unmapped, col].unique()}")
                            df_transformed[col] = df_transformed[col].fillna(df_transformed[col])
                        logger.debug(f"Applied dictionary mapping to column {col}")

                elif callable(mapping):
                    # Function mapping (e.g., for MVDVNAME)
                    if col in df_transformed.columns:
                        df_transformed[col] = df_transformed[col].apply(mapping)
                        logger.debug(f"Applied function mapping to column {col}")

                else:
                    logger.warning(f"Unrecognized mapping type for column {col}: {type(mapping)}")

            except Exception as e:
                logger.error(f"Error processing column {col}: {e}")
                raise

        # Validate transformations
        null_counts = df_transformed.isnull().sum()
        if null_counts.any():
            logger.warning(f"Found null values after transformation:\n{null_counts[null_counts > 0]}")

        logger.info("Successfully completed all transformations")
        return df_transformed

    except Exception as e:
        logger.error(f"An error occurred during transformation: {e}")
        raise

    finally:
        logger.debug("Format gains map function execution completed")


In [6]:
gains_df = format_gains_map(filtered_df)
gains_df.head()

MVDVAPETTAG      9
GAIN-VALUE     225
dtype: int64


,NUMBER,Variable Type,Variable Name,TYPE,MVDVNAME,MVDVAPETTAG,MVDVNUMBER,GAIN-VALUE,GAIN-TAG
0,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,Manipulated Variable,Fresh Feed,PID-J130-1.PV,1.1,0.033053,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
1,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,Manipulated Variable,1 Vibrating Feeder1 OP,PID-J141-LIC-G1118.OP,2.1,-0.529627,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
2,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,Manipulated Variable,1 Vibrating Feeder2 OP,PID-J141-LIC-G1128.OP,2.2,-0.529627,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
3,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,Manipulated Variable,2 Vibrating Feeder1 OP,PID-J142-LIC-G1118.OP,3.1,-0.490164,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...
4,1.1,Average_Bin_Level,BIN8-J140-BIN-01.LEVEL-01.READING,Manipulated Variable,2 Vibrating Feeder2 OP,PID-J142-LIC-G1128.OP,3.2,-0.490164,SIS-JIG.SEP.PWO-SEP_T-CRUSHING.PROFIT_AVERAGE_...


In [7]:
print(gains_df.shape)

(261, 9)
